## Load TEST data

In [67]:
import pandas as pd
import numpy as np
from collections import defaultdict
from pandas import DataFrame

In [68]:
#load positive data id
pos_ids = defaultdict(list)
lines = open("positive_data.txt", 'r').readlines()
for l in lines:
    es = l.strip().split()
    pos_ids[es[0]].append(es[1])
print(pos_ids)
print(np.sum([len(l) for l in pos_ids.values()]))

defaultdict(<class 'list'>, {'A': ['A10', 'B1', 'B2', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'H1', 'H2', 'H3', 'I10', 'J1', 'J2', 'J3', 'K1', 'K3', 'L10', 'M3', 'N2', 'N5', 'P1', 'P2', 'P3', 'Q5', 'Q10'], 'B': ['C7', 'C12', 'H1', 'H2', 'H3', 'I10', 'N2'], 'C': ['H1', 'H2', 'H3'], 'D': ['C4', 'H3'], 'E': ['C4', 'C7', 'M10', 'Q1', 'Q2', 'Q3'], 'F': ['C4', 'D10', 'H1', 'H3', 'M3'], 'G': ['B1', 'C5', 'C7', 'C9', 'I1', 'I3', 'J1', 'M10'], 'H': ['B1', 'E10', 'H4', 'J1', 'L7']})
63


In [69]:
#load energy data
all_data = pd.DataFrame()
for i in ["A","B","C","D","E","F","G","H"]:
    #print(i)
    df = pd.read_csv("ddg/test/U"+i+"_ddg_terms.txt", sep="\t")
    #delete all zero col
    df = df.loc[:,~(df==0).all(axis=0)]
    df.loc[len(df.index)] = ["B1"] + [0.0]*18
    
    df['UAA'] = i
    df['label'] = [t in pos_ids[i] for t in df["MUT"]]
    all_data = pd.concat([all_data, df], axis=0, ignore_index=True)
all_data = all_data.rename(columns={"MUT": "id"})

In [70]:
#remove duplicates
#I12 M11
#L8 I10
#J7 N3
all_data = all_data[all_data['id']!="M11" ]
all_data = all_data[all_data['id']!="I10" ]
all_data = all_data[all_data['id']!="N3" ]

In [71]:
#load ESM-1v, ESM-1b
df1v = pd.read_csv("../evotuning/mm_diskAB_1v_labeled.csv")
df1b = pd.read_csv("../evotuning/mm_diskAB_1b_labeled.csv")

In [72]:
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_1']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_2']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_3']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_4']], how='left', on='id')
all_data = pd.merge(all_data, df1v.loc[:, ['id','esm1v_t33_650M_UR90S_5']], how='left', on='id')
all_data = pd.merge(all_data, df1b.loc[:, ['id','esm_msa1b_t12_100M_UR50S']], how='left', on='id')

In [73]:
all_data

,id,fa_rep:,ref:,hbond_sc:,fa_sol:,yhh_planarity:,dslf_fa13:,fa_intra_sol_xover4:,omega:,p_aa_pp:,...,fa_dun:,rama_prepro:,UAA,label,esm1v_t33_650M_UR90S_1,esm1v_t33_650M_UR90S_2,esm1v_t33_650M_UR90S_3,esm1v_t33_650M_UR90S_4,esm1v_t33_650M_UR90S_5,esm_msa1b_t12_100M_UR50S
0,K12,1.146,-0.043,5.221,0.000,0.000,0.016,0.000,0.000,2.203,...,0.000,0.000,A,False,NaN,NaN,NaN,NaN,NaN,NaN
1,K11,-0.495,3.148,2.325,0.000,0.000,-0.003,0.000,0.000,4.621,...,0.000,0.000,A,False,NaN,NaN,NaN,NaN,NaN,NaN
2,K10,-0.559,0.657,1.911,0.000,0.000,-0.023,0.000,0.000,1.611,...,0.000,0.000,A,False,NaN,NaN,NaN,NaN,NaN,NaN
3,M3,0.126,-0.283,1.720,0.001,-0.002,-0.021,-0.009,-0.003,1.165,...,0.021,-0.001,A,True,NaN,NaN,NaN,NaN,NaN,NaN
4,M2,0.011,-0.537,3.081,0.000,0.000,0.087,0.000,0.000,3.116,...,0.000,0.000,A,False,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,J3,1.142,1.141,-0.732,0.000,0.000,-0.034,0.000,0.000,-2.428,...,0.000,0.000,H,False,NaN,NaN,NaN,NaN,NaN,NaN
484,M10,1.260,-1.203,1.863,-0.112,0.012,0.065,0.002,-0.003,1.063,...,0.125,-0.010,H,False,NaN,NaN,NaN,NaN,NaN,NaN
485,M12,0.161,5.190,-0.319,0.000,0.000,-0.023,0.000,0.000,0.569,...,0.000,0.000,H,False,NaN,NaN,NaN,NaN,NaN,NaN
486,M1,1.873,-1.680,-0.139,0.000,0.000,-0.028,0.000,0.000,-3.994,...,0.000,0.000,H,False,NaN,NaN,NaN,NaN,NaN,NaN


### unirep

In [74]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [75]:
#load eUnirep 1900/256/64 embedding
from jax_unirep.evotuning_models import mlstm64
from jax_unirep.utils import load_params
from jax_unirep import get_reps

evo_tag = "64"
evo_num = int(evo_tag)
params = load_params(folderpath='../evotuning/unirep/weights/'+evo_tag+'/', paper_weights=evo_num)[1]
h_avg, h_final, c_final= get_reps(sequences, params=params,mlstm_size=evo_num)

In [76]:
i2s = {}
lines = open("map_label_fname_test.txt", 'r')
for l in lines:
    es = l.strip().split()
    i2s[es[0]] = es[2]

In [77]:
sequences = [i2s[t] for t in i2s.keys()]
s2t = {i2s[t]:t for t in i2s.keys()}

In [78]:
df_evo = pd.DataFrame()
for i, t in enumerate(i2s.keys()):
    d = pd.Series([s2t[sequences[i]]]+[x for x in h_avg[i]])
    assert(t == s2t[sequences[i]])
    df_evo = pd.concat([df_evo, d.to_frame().T], axis=0, ignore_index=True)

In [79]:
convert = {i:"e"+str(i) for i in range(1+evo_num)}
convert[0] = "id"
df_evo = df_evo.rename(columns=convert)

In [80]:
df_evo

,id,e1,e2,e3,e4,e5,e6,e7,e8,e9,...,e55,e56,e57,e58,e59,e60,e61,e62,e63,e64
0,I1,0.802625,-0.417011,0.220576,0.157626,0.369944,0.075831,0.277032,0.108935,0.021523,...,-0.074041,-0.08568,0.215716,0.62254,-0.133647,0.191179,0.156932,0.077067,-0.027985,0.810587
1,I2,0.818269,-0.330331,0.230842,0.127586,0.321412,0.076229,0.248814,0.115703,0.016251,...,-0.076289,-0.08573,0.222507,0.625803,-0.111198,0.191008,0.157659,0.081806,-0.035592,0.823472
2,I3,0.804438,-0.43444,0.218217,0.150573,0.367343,0.070387,0.272179,0.113096,0.01605,...,-0.064281,-0.084282,0.214313,0.635984,-0.108564,0.18189,0.144754,0.07148,-0.007243,0.824102
3,I4,0.825492,-0.482248,0.216954,0.124416,0.338016,0.083258,0.247668,0.113754,0.02115,...,-0.062744,-0.081333,0.238415,0.6754,-0.115468,0.200942,0.162591,0.077802,-0.078235,0.807159
4,I5,0.811929,-0.447034,0.22245,0.15519,0.35996,0.074246,0.267474,0.11351,0.019262,...,-0.063309,-0.078263,0.218672,0.655193,-0.107883,0.182096,0.147458,0.074843,-0.019423,0.823792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,M10,0.808594,-0.375274,0.224519,0.145749,0.336175,0.085487,0.256601,0.112463,0.017889,...,-0.075949,-0.094025,0.216648,0.634174,-0.107031,0.205505,0.156101,0.084054,-0.05887,0.820763
57,M12,0.832397,-0.425896,0.229339,0.144675,0.290828,0.076643,0.243399,0.119516,0.021988,...,-0.061091,-0.089456,0.234949,0.639827,-0.117317,0.201393,0.153615,0.081044,-0.104383,0.819406
58,N1,0.831263,-0.347303,0.221123,0.195124,0.312048,0.088122,0.255236,0.113181,0.014443,...,-0.050187,-0.095186,0.227386,0.594443,-0.117673,0.208292,0.159842,0.065409,-0.106942,0.823951
59,N2,0.807001,-0.382418,0.224678,0.148746,0.358257,0.080606,0.270925,0.117961,0.017541,...,-0.074216,-0.085782,0.212491,0.631479,-0.111383,0.190567,0.151781,0.071164,-0.048599,0.8224


In [81]:
raw_data = pd.merge(all_data, df_evo.loc[:, :], how='left', on='id')

In [82]:
#extract X
col = raw_data.columns.values.tolist()
#col.remove('tag')
col.remove('UAA')
col.remove('label')
col.remove('id')
col.remove('ref:')
col.remove('dslf_fa13:')
col.remove('omega:')
col.remove('p_aa_pp:')
col.remove('cart_bonded:')
col.remove('rama_prepro:')

In [83]:
import numpy as np
X = np.array(raw_data[col])
y = np.array(raw_data['label'])

In [84]:
print(X.shape, np.sum(y))

(488, 82) 19


In [85]:
#test_set = pd.DataFrame(X_test)
#test_set.insert(test_set.shape[1], "label", y_test)

In [86]:
import pickle

In [87]:
with open("FWD_evo"+evo_tag+"_test.pickle", 'wb') as f:
    pickle.dump([X, y], f)